In [1]:
import time
import math 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re

In [3]:
browser = Browser('edge')

In [4]:
url = "https://www.canadiantire.ca/en/promotions/clearance.html?page=20"
browser.visit(url)

In [5]:
html = browser.html
html_soup = soup(html, 'html.parser')

In [17]:
product_info = html_soup.find_all(class_='nl-product__content')
product_data1 = []

for product in product_info:
    product_name_element = product.find('div', class_='nl-product-card__title')
    original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
    clearance_price_element = product.find('span', class_='nl-price--total--red')
    rating_element = product.find('div', class_='bv_text')
    image_element = product.find('div', class_='nl-product-card__image-wrap')
    anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
    img_tag = product.find('img', attrs={'data-src': True})

    product_name = product_name_element.text.strip() if product_name_element else 'N/A'
    original_price = original_price_element.text.strip() if original_price_element else 'N/A'
    clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
    rating = rating_element.text.strip() if rating_element else None
    product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
    product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
    product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
    product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

    if original_price == 'N/A':
        alt_original_price_element = product.find('span', class_='nl-price--total')
        original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

    clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
    original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

    product_data1.append({
        'Product Name': product_name,
        'Original Price': original_price,
        'Clearance Price': clearance_price,
        'Rating': rating,
        'Product Code': product_code,
        'Link': product_link, 
        'Image Link': product_image_link
    })

print(product_data1)

[{'Product Name': 'CCM FitLite 3DS Hockey Helmet Sticker/Decal Kit', 'Original Price': 9.99, 'Clearance Price': 1.93, 'Rating': '0.0', 'Product Code': '083-8800-0', 'Link': 'https://www.canadiantire.ca/en/pdp/ccm-fitlite-3ds-hockey-helmet-sticker-decal-kit-0838800p.html#srp', 'Image Link': 'https://media-www.canadiantire.ca/product/playing/hockey/hockey-equipment/0838800/ccm-3ds-sticker-kit-18e14de3-1a1b-405a-8b17-7ca7155c99f9.png?im=whresize&wid=142&hei=142'}, {'Product Name': 'KidKusion Gummi Crib Rail Cover', 'Original Price': 21.99, 'Clearance Price': 7.93, 'Rating': '1.5', 'Product Code': '046-8997-2', 'Link': 'https://www.canadiantire.ca/en/pdp/kidkusion-gummi-crib-rail-cover-0468997p.html#srp', 'Image Link': 'https://media-www.canadiantire.ca/product/automotive/car-care-accessories/child-travel-safety/0468997/kidkusion-gummi-crib-rail-cover-a8536c64-4659-4d91-8872-0eb47b2a1819.png?im=whresize&wid=142&hei=142'}, {'Product Name': 'CANVAS Rubberized Ceramic Bathroom/Vanity Tray', '

In [18]:
df1 = pd.DataFrame(product_data1)

df1.sample(24)

,Product Name,Original Price,Clearance Price,Rating,Product Code,Link,Image Link
4,Netgear Wireless AC1750 Dual Band Smart Wi-Fi ...,129.99,49.93,4.7,069-4089-4,https://www.canadiantire.ca/en/pdp/netgear-wir...,https://media-www.canadiantire.ca/product/livi...
7,"Perky Pet Squirrel Stumper Wild Bird Feeder, 3-lb",24.99,14.93,3.2,142-0762-8,https://www.canadiantire.ca/en/pdp/perky-pet-s...,https://media-www.canadiantire.ca/product/livi...
18,"CANVAS Cloud Print Light Filtering Curtain, 2-...",44.99,22.93,4.5,046-6857-8,https://www.canadiantire.ca/en/pdp/canvas-clou...,https://media-www.canadiantire.ca/product/livi...
23,Outbound Youth Marock Insulated PU Leather/Rub...,76.99,14.93,4.1,187-2428X,https://www.canadiantire.ca/en/pdp/outbound-yo...,https://media-www.canadiantire.ca/product/play...
2,CANVAS Rubberized Ceramic Bathroom/Vanity Tray,13.99,7.93,3.3,063-6232-2,https://www.canadiantire.ca/en/pdp/canvas-rubb...,https://media-www.canadiantire.ca/product/livi...
6,ClosetMaid Suite Symphony Laminate Wood Top Sh...,29.99,14.93,3.0,068-7358-8,https://www.canadiantire.ca/en/pdp/closetmaid-...,https://media-www.canadiantire.ca/product/livi...
11,"Arlo Adjustable Indoor and Outdoor Mount, White",24.99,7.93,0.0,046-3281-4,https://www.canadiantire.ca/en/pdp/arlo-adjust...,https://media-www.canadiantire.ca/product/fixi...
15,Berkley Fusion 19 Octopus Hooks,5.29,2.43,4.8,077-7717X,https://www.canadiantire.ca/en/pdp/berkley-fus...,https://media-www.canadiantire.ca/product/play...
1,KidKusion Gummi Crib Rail Cover,21.99,7.93,1.5,046-8997-2,https://www.canadiantire.ca/en/pdp/kidkusion-g...,https://media-www.canadiantire.ca/product/auto...
20,Plano 3700 Guide Series Stowaway,19.99,13.93,4.5,078-5922-2,https://www.canadiantire.ca/en/pdp/plano-3700-...,false?im=whresize&wid=142&hei=142


In [30]:
df1['Percentage Off'] = round((df1['Clearance Price']/df1['Original Price'])*100)

df1.sample(24)

,Product Name,Original Price,Clearance Price,Rating,Product Code,Link,Image Link,Product Category,Percentage Off
19,Petco Plush Giant Rat Cat Toy with Catnip,5.99,3.93,4.3,142-3855-0,https://www.canadiantire.ca/en/pdp/petco-plush...,https://media-www.canadiantire.ca/product/livi...,pet-care,66.0
0,CCM FitLite 3DS Hockey Helmet Sticker/Decal Kit,9.99,1.93,0.0,083-8800-0,https://www.canadiantire.ca/en/pdp/ccm-fitlite...,https://media-www.canadiantire.ca/product/play...,hockey,19.0
17,"SIMONIZ Waterless Car Wash & Wax Spray, 454-g",13.99,6.93,4.5,039-2520-0,https://www.canadiantire.ca/en/pdp/simoniz-wat...,https://media-www.canadiantire.ca/product/auto...,car-care-accessories,50.0
6,ClosetMaid Suite Symphony Laminate Wood Top Sh...,29.99,14.93,3.0,068-7358-8,https://www.canadiantire.ca/en/pdp/closetmaid-...,https://media-www.canadiantire.ca/product/livi...,home-organization,50.0
1,KidKusion Gummi Crib Rail Cover,21.99,7.93,1.5,046-8997-2,https://www.canadiantire.ca/en/pdp/kidkusion-g...,https://media-www.canadiantire.ca/product/auto...,car-care-accessories,36.0
16,Browning Windshield Protector,22.99,7.93,3.0,037-0089-4,https://www.canadiantire.ca/en/pdp/browning-wi...,https://media-www.canadiantire.ca/product/auto...,car-care-accessories,34.0
2,CANVAS Rubberized Ceramic Bathroom/Vanity Tray,13.99,7.93,3.3,063-6232-2,https://www.canadiantire.ca/en/pdp/canvas-rubb...,https://media-www.canadiantire.ca/product/livi...,home-decor,57.0
4,Netgear Wireless AC1750 Dual Band Smart Wi-Fi ...,129.99,49.93,4.7,069-4089-4,https://www.canadiantire.ca/en/pdp/netgear-wir...,https://media-www.canadiantire.ca/product/livi...,electronics,38.0
14,Brother TZE335 White on Black Laminated Label ...,29.99,17.93,0.0,069-4789-8,https://www.canadiantire.ca/en/pdp/brother-tze...,https://media-www.canadiantire.ca/product/livi...,electronics,60.0
18,"CANVAS Cloud Print Light Filtering Curtain, 2-...",44.99,22.93,4.5,046-6857-8,https://www.canadiantire.ca/en/pdp/canvas-clou...,https://media-www.canadiantire.ca/product/livi...,home-decor,51.0


In [31]:
from urllib.parse import urlparse

def extract_category(link_url):
    parsed_url = urlparse(link_url)
    path_components = parsed_url.path.split('/')
    category = '/'.join(path_components[3:4])
    return category

df1['Product Category'] = df1['Image Link'].apply(extract_category)

df1

,Product Name,Original Price,Clearance Price,Rating,Product Code,Link,Image Link,Product Category,Percentage Off
0,CCM FitLite 3DS Hockey Helmet Sticker/Decal Kit,9.99,1.93,0.0,083-8800-0,https://www.canadiantire.ca/en/pdp/ccm-fitlite...,https://media-www.canadiantire.ca/product/play...,hockey,19.0
1,KidKusion Gummi Crib Rail Cover,21.99,7.93,1.5,046-8997-2,https://www.canadiantire.ca/en/pdp/kidkusion-g...,https://media-www.canadiantire.ca/product/auto...,car-care-accessories,36.0
2,CANVAS Rubberized Ceramic Bathroom/Vanity Tray,13.99,7.93,3.3,063-6232-2,https://www.canadiantire.ca/en/pdp/canvas-rubb...,https://media-www.canadiantire.ca/product/livi...,home-decor,57.0
3,"For Living Acrylic Toothbrush Holder, Clear",8.99,4.93,4.7,063-2245-6,https://www.canadiantire.ca/en/pdp/for-living-...,https://media-www.canadiantire.ca/product/livi...,home-decor,55.0
4,Netgear Wireless AC1750 Dual Band Smart Wi-Fi ...,129.99,49.93,4.7,069-4089-4,https://www.canadiantire.ca/en/pdp/netgear-wir...,https://media-www.canadiantire.ca/product/livi...,electronics,38.0
5,"Hillman Mirror Clips, Decorative, 20-lb Capaci...",2.99,0.93,4.0,161-0729-4,https://www.canadiantire.ca/en/pdp/hillman-mir...,https://media-www.canadiantire.ca/product/fixi...,hardware,31.0
6,ClosetMaid Suite Symphony Laminate Wood Top Sh...,29.99,14.93,3.0,068-7358-8,https://www.canadiantire.ca/en/pdp/closetmaid-...,https://media-www.canadiantire.ca/product/livi...,home-organization,50.0
7,"Perky Pet Squirrel Stumper Wild Bird Feeder, 3-lb",24.99,14.93,3.2,142-0762-8,https://www.canadiantire.ca/en/pdp/perky-pet-s...,https://media-www.canadiantire.ca/product/livi...,pet-care,60.0
8,"Zebco Rhino Spincast Fishing Reel, Anti-Revers...",54.99,29.93,1.9,077-8433-0,https://www.canadiantire.ca/en/pdp/zebco-rhino...,https://media-www.canadiantire.ca/product/play...,fishing,54.0
9,For Living Stainless Steel Soap Dispenser,12.99,7.73,3.3,063-2295-6,https://www.canadiantire.ca/en/pdp/for-living-...,https://media-www.canadiantire.ca/product/livi...,home-decor,60.0


In [32]:
base_url = "https://www.canadiantire.ca/en/promotions/clearance.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(base_url)
time.sleep(5)

html_content = driver.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver.page_source
    html_soup = soup(html_content, 'html.parser')

    product_info = html_soup.find_all(
        class_='nl-product-card__primary-details')

    for product in product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

In [ ]:
clearance_df = pd.DataFrame(product_data)
clearance_df['Percentage Off'] = round((clearance_df['Clearance Price']/clearance_df['Original Price'])*100)
clearance_df['Product Category'] = clearance_df['Image Link'].apply(extract_category)
clearance_df.info

In [ ]:
base_url = "https://www.canadiantire.ca/en/promotions/hot-sale.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver2 = webdriver.Chrome(service=service)

driver2.get(base_url)
time.sleep(5)
html_content = driver2.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver2.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver2.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info = html_soup.find_all(class_='nl-product-card__primary-details')

    for product in sale_product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver2.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

In [ ]:
sale_df = pd.DataFrame(sale_product_data)
sale_df['Percentage Off'] = round((sale_df['Sale Price']/sale_df['Original Price'])*100)
sale_df['Product Category'] = sale_df['Image Link'].apply(extract_category)
sale_df.info

In [ ]:
base_url = "https://www.canadiantire.ca/en/promotions/red-alert-deals.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver3 = webdriver.Chrome(service=service)

driver3.get(base_url)
time.sleep(5)
html_content = driver3.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data_2 = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver3.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver3, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver3.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info2 = html_soup.find_all(class_='nl-product-card__primary-details')

    for product in sale_product_info2:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data_2.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver3.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

In [ ]:
sale_df2 = pd.DataFrame(sale_product_data_2)
sale_df['Percentage Off'] = round((sale_df['Sale Price']/sale_df['Original Price'])*100)
sale_df['Product Category'] = sale_df['Image Link'].apply(extract_category)
sale_df.info

In [ ]:
clearance_df.sample(25)

In [ ]:
sale_df.sample(25)

In [ ]:
sale_df2.sample(25)

In [ ]:
clearance_df.to_csv("./clearance.csv", index=True)
sale_df.to_csv("./sale.csv", index=True)
sale_df2.to_csv("./sale2.csv", index=True)